# 🎲 The Black-Jack — *Classic Before Quantum*  

Hello everyone! 👋  

Before diving into **Quantum Blackjack**, it’s good practice to build the game in its **classical form** first.  
In this notebook, we’ll walk through how to implement Blackjack step by step — the traditional way 🃏


### 🃏 Blackjack (a.k.a. 21) — Quick Rules

Welcome to Blackjack — the game where you try to hit 21 without exploding 💥  
No chips, no money, just pure card chaos 🔮

---

### 🎯 Objective
- Get as close to **21** as you can.  
- If you go **over 21**, you **bust 💥** (instant loss).  
- If the dealer busts and you don’t, you win 🙌.  
- Otherwise, whoever is **closer to 21** wins.

---

### 🧮 Card Values
- **2–10** → face value (7 = 7, 9 = 9, etc.)  
- **J, Q, K** → all worth **10 👑**  
- **Ace (A)** → **1 or 11** (whichever helps you more 🦸)

In [2]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def value(self):
        try:
            if self.rank in ['J', 'Q', 'K']:
                return 10
            if self.rank == 'A':
                return 11  # or 1 based on game rules
            return int(self.rank)  # Assuming rank is a number string for 2-10
        except ValueError:
            raise ValueError("Invalid rank value")



#PROGRAM TEST
card = Card(4, 'Hearts')
print(card.value())  # Should print 4

4


In [12]:
import random

class Deck:
    def __init__(self, n_decks=1):
        self.cards = []
        for _ in range(n_decks):
            for suit in ['Hearts', 'Diamonds', 'Clubs', 'Spades']:
                for rank in range(2, 11):
                    self.cards.append(Card(rank, suit))
                for rank in ['J', 'Q', 'K', 'A']:
                    self.cards.append(Card(rank, suit))
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def draw(self):
        return self.cards.pop() if self.cards else None
    

#PROGRAM TEST
deck = Deck()
print(len(deck.cards))  # Should print 52 for a single deck
card = deck.draw()
print(card.rank, card.suit)  # Should print the rank and suit of the drawn card

52
4 Diamonds



### 🃏 The Deal
- You start with **2 cards**.  
- Dealer also gets **2 cards**: one face-up 👀, one face-down 🤫.


In [16]:
class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def best_value(self):
        # Calculate the best total value of the hand, considering Aces as 1 or 11
        total = sum(card.value() for card in self.cards)
        aces = sum(1 for card in self.cards if card.rank == 'A')
        while total > 21 and aces:
            total -= 10
            aces -= 1
        return total

    def is_blackjack(self):
        return len(self.cards) == 2 and self.best_value() == 21

    def is_bust(self):
        return self.best_value() > 21


#PROGRAM TEST
hand = Hand()
hand.add_card(Card('A', 'Hearts'))
hand.add_card(Card(10, 'Diamonds'))
hand.add_card(Card(10, 'Clubs'))
print(hand.best_value())  # Should print 21

21



### 💡 Your Choices
- **Hit ➕** → take another card.  
- **Stand ✋** → stop and lock in your total.  
- _(Optional extras you can add later: Double, Split, etc.)_


In [26]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

    def draw(self, deck):
        card = deck.draw()
        if card:
            self.hand.add_card(card)
        return card

    def decide(self):
        # Ask for input (hit/stand)
        action = input(f"{self.name}, do you want to hit or stand? ").strip().lower()
        while action not in ['hit', 'stand']:
            print("Invalid input. Please enter 'hit' or 'stand'.")
            action = input(f"{self.name}, do you want to hit or stand? ").strip().lower()
        return action


#PROGRAM TEST
player = Player("Alice")
deck = Deck()
player.draw(deck)
print(player.hand.cards[0].rank, player.hand.cards[0].suit)  # Should print the rank and suit of the drawn card
player.decide()  # Should prompt for input

J Diamonds


'hit'

### 🤖 Dealer’s Play
- Dealer keeps drawing until total is **17 or higher**.  
- Dealer rule (we’ll use this one ✅):  
  - **S17** → dealer *stands* on soft 17 (Ace+6).  


In [39]:
class Dealer(Player):
    def play(self, deck):
        # Dealer keeps hitting until 17 or higher
        while self.hand.best_value() < 17:
            self.draw(deck)
            print(f"{self.name} drew a card.")


#PROGRAM TEST
dealer = Dealer("Dealer")
dealer.play(deck)
for card in dealer.hand.cards:
    print(card.rank, card.suit)  # Should print all cards in dealer's hand

Dealer drew a card.
Dealer drew a card.
Dealer drew a card.
Dealer drew a card.
8 Spades
2 Hearts
5 Diamonds
3 Hearts


### ✨ Special Case
- **Blackjack!** (Ace + 10-value card on the first deal)  
  → That’s the best possible hand, game over 🎉

---

### 📝 Terms
- **Soft hand** → has an Ace counted as 11 (e.g., A+6 = soft 17).  
- **Hard hand** → no Ace or Ace counted as 1.  
- **Bust** → total over 21 = you’re out 💥.



In [ ]:
class Game:
    def __init__(self):
        self.deck = Deck()
        self.deck.shuffle()
        self.player = Player("Player")
        self.dealer = Dealer("Dealer")

    def deal_initial(self):
        for _ in range(2):
            card = self.player.draw(self.deck)
            print(f"{self.player.name} drew {card.rank} of {card.suit}. Current value: {self.player.hand.best_value()}")
            self.dealer.draw(self.deck)
        print(f"{self.player.name}'s initial hand value: {self.player.hand.best_value()}")
        print(f"{self.dealer.name}'s visible card: {self.dealer.hand.cards[0].rank} of {self.dealer.hand.cards[0].suit}")
        if self.player.hand.is_blackjack():
            print(f"{self.player.name} has a blackjack! {self.player.name} wins!")

    def player_turn(self):
        action = self.player.decide()
        while action == 'hit':
            card = self.player.draw(self.deck)
            print(f"{self.player.name} drew {card.rank} of {card.suit}. Current value: {self.player.hand.best_value()}")
            if self.player.hand.is_bust():
                print(f"{self.player.name} busts with value {self.player.hand.best_value()}!")
                return
            action = self.player.decide()
        # Player stands
        print(f"{self.player.name} stands with value {self.player.hand.best_value()}.")

    def dealer_turn(self):
        self.dealer.play(self.deck)
        print(f"{self.dealer.name} stands with value {self.dealer.hand.best_value()}.")

    def determine_winner(self):
        player_value = self.player.hand.best_value()
        dealer_value = self.dealer.hand.best_value()

        if self.player.hand.is_bust():
            print("Dealer wins! Player busted.")
        elif self.dealer.hand.is_bust():
            print("Player wins! Dealer busted.")
        elif player_value > dealer_value:
            print("Player wins!")
        elif dealer_value > player_value:
            print("Dealer wins!")
        else:
            print("It's a tie!")

    def play_round(self):
        self.player_turn()
        if not self.player.hand.is_bust():
            self.dealer_turn()

    def start(self):
        self.deal_initial()
        if not self.player.hand.is_blackjack():
            self.play_round()
            self.determine_winner()

game = Game()
game.start()

After filling / Implementing each above our game loop will look like:


```python
# 🎮 Running the Game

def main():
    game = Game()  # create a new game
    
    while True:
        print("\n--- New Round ---")
        game.play_round()  # one full round of Blackjack
        
        # Ask player if they want to play again
        again = input("Play again? (y/n): ").lower()
        if again != "y":
            print("Thanks for playing! 👋")
            break


# Start the game loop
if __name__ == "__main__":
    main()
